# Sensitivity analysis for OBG early detection project

This notebook is designed to help perform a sensitivity analysis for QALY outcomes across a range of values for several parameters in pathosim.

In [ ]:
import pathosim as inf
import sciris as sc
import numpy as np

##### Inputs

Life expectancy table for the UK population (taken from National life tables 2020 to 2022, Office for National Statistics). We use the expected residual life (remaining years) at a given age (from age 0 to 100). This allows us to measure the burden of disease in metrics such as Years of Life Lost (YLL).

In [ ]:
le_UK = inf.LifeExpectancy(
    years_remaining_males=[
        78.57, 77.91, 76.93, 75.94, 74.95, 73.96, 72.96, 71.97, 70.97, 69.98,
        68.98, 67.99, 66.99, 66.00, 65.01, 64.01, 63.02, 62.04, 61.06, 60.08,
        59.11, 58.14, 57.17, 56.20, 55.23, 54.26, 53.29, 52.33, 51.36, 50.39,
        49.43, 48.47, 47.51, 46.55, 45.60, 44.65, 43.69, 42.75, 41.81, 40.86,
        39.93, 39.00, 38.07, 37.14, 36.22, 35.30, 34.39, 33.48, 32.58, 31.69,
        30.80, 29.92, 29.04, 28.17, 27.30, 26.43, 25.58, 24.73, 23.89, 23.05,
        22.23, 21.41, 20.61, 19.81, 19.03, 18.25, 17.48, 16.73, 15.99, 15.26,
        14.54, 13.84, 13.14, 12.46, 11.78, 11.12, 10.47, 9.86, 9.26, 8.68,
        8.12, 7.60, 7.08, 6.60, 6.12, 5.68, 5.25, 4.85, 4.48, 4.14, 3.81,
        3.52, 3.25, 3.00, 2.78, 2.57, 2.40, 2.22, 2.07, 1.91, 1.8
],
    years_remaining_females=[
        82.57, 81.86, 80.88, 79.89, 78.90, 77.90, 76.91, 75.91, 74.92, 73.92,
        72.92, 71.93, 70.93, 69.94, 68.94, 67.95, 66.96, 65.97, 64.98, 63.99,
        63.00, 62.01, 61.03, 60.04, 59.06, 58.07, 57.08, 56.10, 55.12, 54.13,
        53.15, 52.17, 51.19, 50.21, 49.24, 48.27, 47.30, 46.33, 45.36, 44.40,
        43.44, 42.48, 41.53, 40.58, 39.63, 38.68, 37.74, 36.80, 35.87, 34.94,
        34.02, 33.10, 32.18, 31.27, 30.36, 29.46, 28.56, 27.66, 26.78, 25.89,
        25.02, 24.15, 23.29, 22.44, 21.60, 20.76, 19.93, 19.12, 18.31, 17.51,
        16.72, 15.94, 15.17, 14.41, 13.66, 12.92, 12.20, 11.51, 10.83, 10.16,
        9.53, 8.91, 8.32, 7.75, 7.20, 6.67, 6.17, 5.70, 5.26, 4.85, 4.47, 4.11,
        3.78, 3.49, 3.23, 2.97, 2.74, 2.54, 2.35, 2.20, 2.04
])

##### Baseline simulation parameters

In [ ]:
sim_pars = dict(
        use_waning    = True,           
        pop_size      = 5000,       
        pop_type      = 'behaviour_module',      
        n_days        = 100,            
        verbose       = 0,             
        rand_seed     = 42,
        burden        = inf.Burden(life_expectancy=le_UK) # this will compute burden/utility statistics and report them in results and summary
    ) 

##### Hyperparameters

In [ ]:
n = 5   # Number of simulations for each scenario
maxcpu = 0.9
maxmem = 0.9
parallelize = False

##### Sensitivity analysis parameters

In [ ]:
ttd = [4.2, 7.0, 9.7]                      # Time to detection
response = ['weak', 'medium', 'strong']
ttr = [0, 0, 0]                            # Time to response
ifr = [0.004, 0.009, 0.014]                # Infection fatality rate (IFR)
incubation = [0.6, 4.0, 5.6]               # Incubation period
beta = [0.91, 2.3, 4.5]                    # Transmission rate

##### Pathogens

In [ ]:
pathogens = []

for x in ifr:
    for y in incubation:
        for y in beta:
            pathogen = inf.SARS_COV_2(10)
            # Configure pathogen according to x, y and z
            pathogen.configure_transmission(beta = y) 
            pathogens.append(pathogen)

##### Interventions

In [ ]:
# intervention packages (stored as dict)
packages = {
    "weak"   : [inf.intervention_bucket(strength='weak')],
    "medium" : [inf.intervention_bucket(strength='medium')],
    "strong" : [inf.intervention_bucket(strength='strong')]
}

##### Events

In [ ]:
events = [inf.DelayedTrigger('social distancing', start_delay=1, stop_delay=None, trigger_event='detection')]

##### Set up counterfactual simulations

In [ ]:
for pathogen in pathogens:
    for t in ttr:
        # Set up response event according to ttr
        events = [inf.DelayedTrigger('social distancing', start_delay=t, stop_delay=None, trigger_event='detection')]

        # Set up simulations
        cf = inf.CounterfactualMultiSim(sim_pars, pathogens = [pathogen], events=all_events, intervention_packages=packages, n_sims=n, maxcpu=maxcpu, maxmem = maxmem, parallelize = parallelize)